# Pre_tools
Here are some tools for data arrangement. <br>
## Tool 1: Sort files
### Libraries

In [31]:
import time, random
import os, glob, ntpath
import re
import logging
from shutil import move
from datetime import datetime as ddt

import numpy as np
import pandas as pd
import pydicom as dicom

from lxml import etree
from tqdm import tqdm

### Tool 1-1: Sort scattered file
 Sort by study date if files are scattered in patient folder. <br>
 Change `[patient number]/[xxx.dcm]` to `[patient number]/[study date]/xxx.dcm`

In [49]:
source_path = '/home/d/pancreas/raw_data/tumor55'

In [50]:
st_tol = time.time()
cnt = 0
for file in tqdm(glob.glob(source_path+'/*/*.dcm')):
    try:
        date = str(dicom.read_file(file, force = True)[0x0008, 0x0020].value)
        parent_path = os.path.dirname(file) + '/' + date
        if not os.path.exists(parent_path):
            os.makedirs(parent_path)
        move(file, parent_path + '/')
        cnt += 1
    except:
        continue
print('Done moving {} data in {} seconds'.format(cnt, time.time()-st_tol))

100%|██████████| 1971/1971 [00:21<00:00, 91.46it/s]

Done moving 1971 data in 21.568865537643433 seconds


### Tool 1-2: Sort series
Change `[patient number]/[study date]/xxx.dcm` to `[patient number]/[study date]/[series number]/xxx.dcm`

In [124]:
source_path = '/home/d/pancreas/raw_data/tumor55'

In [126]:
st_tol = time.time()
cnt = 0
error_file = []
for case_path in tqdm(glob.glob(source_path + '/00*/*')):
    for file in glob.glob(case_path+'/*.dcm'):
        try:
            series_number = str(dicom.read_file(file, force = True)[0x0020, 0x0011].value)
            parent_path = os.path.dirname(file) + '/' + series_number
            if not os.path.exists(parent_path):
                os.makedirs(parent_path)
            move(file, parent_path + '/')
            cnt +=1 
        except:
            error_file.append(file)
            continue
print('Done moving {} data in {} seconds'.format(cnt, time.time()-st_tol))

100%|██████████| 55/55 [05:39<00:00,  6.18s/it]

Done moving 27941 data in 340.0053536891937 seconds


In [127]:
print(error_file)

[]


### Tool: Pending
Save as series discription <br>
Problem: can't find "series name", some series discription are too long, empty, or garbled

In [116]:
st_tol = time.time()
cnt = 0
for case_path in tqdm(glob.glob(source_path + '/00*/*')):
    for file in glob.glob(case_path+'/*.dcm'):
        try:
            series_name = str(dicom.read_file(file, force = True)[0x0008, 0x103e].value)
            series_name = ' '.join(series_name.split()).strip().replace(' ', '_')
            if len(name) < 20 and len(name) > 0:
                parent_path = os.path.dirname(file) + '/' + series_name
                if not os.path.exists(parent_path):
                    os.makedirs(parent_path)
                move(file, parent_path + '/')
                cnt += 1
            elif len(series_name) == 0 or len(series_name) >= 20:
                series_number = str(dicom.read_file(file, force = True)[0x0020, 0x0011].value)
                parent_path = os.path.dirname(file) + '/' + series_number
                if not os.path.exists(parent_path):
                    os.makedirs(parent_path)
                move(file, parent_path + '/')
                cnt += 1
        except:
            continue
print('Done moving {} data in {} seconds'.format(cnt, time.time()-st_tol))

100%|██████████| 55/55 [00:00<00:00, 61.95it/s]

Done moving 37 data in 0.9005215167999268 seconds
